In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
pd.set_option("display.max_columns",101)

In [9]:
game_2009 = pd.read_csv('season_games/games_2009.csv')
game_2010 = pd.read_csv('season_games/games_2010.csv')
game_2011 = pd.read_csv('season_games/games_2011.csv')
game_2012 = pd.read_csv('season_games/games_2012.csv')
game_2013 = pd.read_csv('season_games/games_2013.csv')
game_2014 = pd.read_csv('season_games/games_2014.csv')
game_2015 = pd.read_csv('season_games/games_2015.csv')
game_2016 = pd.read_csv('season_games/games_2016.csv')
game_2017 = pd.read_csv('season_games/games_2017.csv')

In [97]:
all_games = pd.concat([game_2009, game_2010, 
                       game_2011, game_2012, 
                       game_2013, game_2014,
                       game_2015, game_2016], axis=0)

In [98]:
all_games.index = all_games['GameID']

In [79]:
game_player_stats_rush = pd.read_csv('game_player_stats/game_rushing_df.csv')

In [81]:
players = list(game_player_stats_rush['Player_Name'].unique())
game_player_stats_rush['PlayerID'] = pd.Series()
for idx, row in game_player_stats_rush.iterrows():
    game_player_stats_rush.loc[idx, 'PlayerID'] = players.index(row['Player_Name'])

In [83]:
game_player_stats_rush.drop('TD_to_Fumbles', axis=1, inplace=True)

In [91]:
gameid_group = game_player_stats_rush.groupby(['GameID', 'Player_Name']).sum()

In [94]:
players_group = game_player_stats_rush.groupby(['Player_Name', 'Opponent']).sum()

In [101]:
joined1 = gameid_group.join(all_games, how='inner')

In [109]:
joined1.head()

Carries  Drives  Car_per_Drive  Total_Yards  \
GameID     Player_Name                                                     
2009091000 B.Roethlisberger        1       1          1.000            3   
           C.Johnson              15       8          1.875           57   
           K.Collins               2       1          2.000            1   
           L.White                 8       4          2.000           28   
           M.Moore                 5       2          2.500            8   

                             Yards_per_Car  Yards_per_Drive  Fumbles  TDs  \
GameID     Player_Name                                                      
2009091000 B.Roethlisberger            3.0            3.000        0    0   
           C.Johnson                   3.8            7.125        0    0   
           K.Collins                   0.5            1.000        0    0   
           L.White                     3.5            7.000        0    0   
           M.Moore                     1.6            4.000        0    0   

                             Total_EPA  Success_Rate  EPA_per_Car  EPA_Ratio  \
GameID     Player_Name                                                         
2009091000 B.Roethlisberger  -0.214375          0.00    -0.214375   0.000000   
           C.Johnson         -1.273019          0.20    -0.084868   0.452852   
           K.Collins         -0.563014          0.00    -0.281507   0.000000   
           L.White           -1.388659          0.25    -0.173582   0.173200   
           M.Moore           -2.498237          0.20    -0.499647   0.014808   

                             TD_per_Car  Fumbles_per_Car  Fumbles_per_Drive  \
GameID     Player_Name                                                        
2009091000 B.Roethlisberger         0.0              0.0                0.0   
           C.Johnson                0.0              0.0                0.0   
           K.Collins                0.0              0.0                0.0   
           L.White                  0.0              0.0                0.0   
           M.Moore                  0.0              0.0                0.0   

                             TD_Drive  EPA_per_Drive  Total_WPA  \
GameID     Player_Name                                            
2009091000 B.Roethlisberger       0.0      -0.214375  -0.008178   
           C.Johnson              0.0      -0.159127  -0.042698   
           K.Collins              0.0      -0.563014  -0.107496   
           L.White                0.0      -0.347165  -0.051114   
           M.Moore                0.0      -1.249119  -0.141199   

                             WPA_per_Drive  Win_Success_Rate  WPA_per_Car  \
GameID     Player_Name                                                      
2009091000 B.Roethlisberger      -0.008178          0.000000    -0.008178   
           C.Johnson             -0.005337          0.266667    -0.002847   
           K.Collins             -0.107496          0.000000    -0.053748   
           L.White               -0.012778          0.250000    -0.006389   
           M.Moore               -0.070599          0.200000    -0.028240   

                             WPA_Ratio  Total_Clutch_EPA  Clutch_EPA_per_Car  \
GameID     Player_Name                                                         
2009091000 B.Roethlisberger   0.000000         -0.001753           -0.001753   
           C.Johnson          0.452460          0.383406            0.025560   
           K.Collins          0.000000         -0.042151           -0.021075   
           L.White            0.145885         -0.020208           -0.002526   
           M.Moore            0.055183         -0.148918           -0.029784   

                             Clutch_EPA_per_Drive  PlayerID      GameID  \
GameID     Player_Name                                                    
2009091000 B.Roethlisberger             -0.001753       6.0  2009091000   
           C.Johnson                     0.04

In [111]:
# joined1.apply(lambda x: sorted(list([x['home'], x['away']])), axis=1)

ValueError: Shape of passed values is (15427, 2), indices imply (15427, 33)

In [19]:
X_train = pd.merge(game_player_stats_rush, pd.concat([game_2009, game_2010, game_2011, 
                                                       game_2012, game_2013, game_2014, 
                                                       game_2015, game_2016], axis=0), how='inner', on='GameID')
y_train = X_train[['Total_Yards']]
X_train = X_train.drop('Total_Yards', axis=1)

X_test = pd.merge(game_player_stats_rush, game_2017, how='inner', on='GameID')
y_test = X_test[['Total_Yards']]
X_test = X_test.drop('Total_Yards', axis=1)

In [20]:
X_train.describe()

/Applications/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4392: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,GameID,Carries,Drives,Car_per_Drive,Yards_per_Car,Yards_per_Drive,Fumbles,TDs,TD_to_Fumbles,Total_EPA,Success_Rate,EPA_per_Car,EPA_Ratio,TD_per_Car,Fumbles_per_Car,Fumbles_per_Drive,TD_Drive,EPA_per_Drive,Total_WPA,WPA_per_Drive,Win_Success_Rate,WPA_per_Car,WPA_Ratio,Total_Clutch_EPA,Clutch_EPA_per_Car,Clutch_EPA_per_Drive,homescore,awayscore,Season
count,1.571900e+04,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.00000,4163.000000,15719.000000,15719.000000,15719.000000,15694.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15703.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000,15719.000000
mean,2.012629e+09,6.812329,3.756028,1.566896,4.552617,6.986788,0.124245,0.21350,inf,-0.428483,0.398764,-0.059563,0.435660,0.033977,0.038196,0.046118,0.050567,-0.093484,-0.003045,-0.000322,0.424671,-0.000253,0.458487,0.006473,-0.001020,-0.000476,23.758254,21.211655,2012.484636
std,2.291985e+06,6.893572,2.864961,0.718797,4.745704,6.294763,0.365028,0.49796,NaN,2.832631,0.329035,0.895244,0.358303,0.122670,0.155152,0.169627,0.147526,1.039980,0.085901,0.029939,0.330292,0.025944,0.366895,0.273688,0.093110,0.107423,10.521228,10.045630,2.291157
min,2.009091e+09,1.000000,1.000000,1.000000,-34.000000,-34.000000,0.000000,0.00000,0.000000,-17.624733,0.000000,-10.793161,0.000000,0.000000,0.000000,0.000000,0.000000,-10.793161,-1.153846,-0.399023,0.000000,-0.399023,0.000000,-3.951106,-3.393879,-3.393879,0.000000,0.000000,2009.000000
25%,2.011010e+09,1.000000,1.000000,1.000000,2.210526,3.000000,0.000000,0.00000,0.000000,-1.672975,0.125000,-0.375616,0.033172,0.000000,0.000000,0.000000,0.000000,-0.551909,-0.035984,-0.012380,0.166667,-0.008652,0.049262,-0.029737,-0.006924,-0.010070,17.000000,14.000000,2010.000000
50%,2.012123e+09,4.000000,3.000000,1.333333,3.875000,6.000000,0.000000,0.00000,NaN,-0.326979,0.357143,-0.088998,0.412421,0.000000,0.000000,0.000000,0.000000,-0.136246,-0.002807,-0.001162,0.400000,-0.000752,0.447999,-0.000080,-0.000042,-0.000062,23.000000,20.000000,2012.000000
75%,2.014123e+09,10.000000,6.000000,2.000000,5.800000,9.857143,0.000000,0.00000,inf,0.969441,0.545455,0.248850,0.721089,0.000000,0.000000,0.000000,0.000000,0.374351,0.030024,0.010708,0.600000,0.006894,0.777277,0.049613,0.009226,0.015516,31.000000,27.000000,2014.000000
max,2.017010e+09,39.000000,15.000000,13.000000,80.000000,80.000000,4.000000,4.00000,inf,17.982397,1.000000,6.931749,1.000000,1.000000,1.000000,1.500000,1.000000,6.931749,0.666913,0.356942,1.000000,0.275955,1.000000,3.978336,1.576288,1.576288,62.000000,59.000000,2016.000000


In [21]:
list_of_cols_to_drop = ['TD_to_Fumbles']

In [22]:
def dropping_unnecessary_cols(df, list_of_columns):
    df.drop(list_of_columns, axis=1, inplace=True)
    return df

In [23]:
X_train = dropping_unnecessary_cols(X_train, list_of_cols_to_drop)

In [33]:
X_train_num = X_train_num.fillna(0)

In [24]:
X_test = dropping_unnecessary_cols(X_test, list_of_cols_to_drop)

In [26]:
X_train_num = X_train.select_dtypes([np.number])

In [27]:
X_test_num = X_test.select_dtypes([np.number])

In [28]:
rf = RandomForestRegressor()

In [39]:
rf.fit(X_train_num, y_train['Total_Yards'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [36]:
rf.predict(X_test_num.fillna(0))

array([ 151.3,   45. ,   38. , ...,    9. ,    8. ,    3. ])

In [41]:
X_test

,GameID,Rusher_ID,Team,Opponent,Player_Name,Carries,Drives,Car_per_Drive,Yards_per_Car,Yards_per_Drive,Fumbles,TDs,Total_EPA,Success_Rate,EPA_per_Car,EPA_Ratio,TD_per_Car,Fumbles_per_Car,Fumbles_per_Drive,TD_Drive,EPA_per_Drive,Total_WPA,WPA_per_Drive,Win_Success_Rate,WPA_per_Car,WPA_Ratio,Total_Clutch_EPA,Clutch_EPA_per_Car,Clutch_EPA_per_Drive,date,home,away,homescore,awayscore,Season
0,2017090700,00-0033923,KC,NE,K.Hunt,17,11,1.545455,8.705882,13.454545,1,1,0.228444,0.529412,0.013438,0.505405,0.058824,0.058824,0.090909,0.090909,0.020768,0.111301,0.010118,0.529412,0.006547,0.611945,-0.118762,-0.006986,-0.010797,2017-09-07,NE,KC,27,42,2017
1,2017090700,00-0030433,NE,KC,M.Gillislee,15,8,1.875000,3.000000,5.625000,0,3,-4.486024,0.466667,-0.299068,0.338033,0.200000,0.000000,0.000000,0.375000,-0.560753,-0.097204,-0.012151,0.466667,-0.006480,0.379874,-0.440201,-0.029347,-0.055025,2017-09-07,NE,KC,27,42,2017
2,2017090700,00-0031062,NE,KC,J.White,10,4,2.500000,3.800000,9.500000,0,0,0.579506,0.400000,0.057951,0.541818,0.000000,0.000000,0.000000,0.000000,0.144876,0.004152,0.001038,0.600000,0.000415,0.509693,0.000495,0.000050,0.000124,2017-09-07,NE,KC,27,42,2017
3,2017090700,00-0028237,NE,KC,C.Hogan,3,2,1.500000,5.666667,8.500000,0,0,-0.549636,0.333333,-0.183212,0.304851,0.000000,0.000000,0.000000,0.000000,-0.274818,-0.018439,-0.009219,0.333333,-0.006146,0.296367,-0.020036,-0.006679,-0.010018,2017-09-07,NE,KC,27,42,2017
4,2017090700,00-0030288,NE,KC,R.Burkhead,3,2,1.500000,5.000000,7.500000,0,0,0.514194,0.333333,0.171398,0.588414,0.000000,0.000000,0.000000,0.000000,0.257097,0.036142,0.018071,0.333333,0.012047,0.680598,0.096895,0.032298,0.048448,2017-09-07,NE,KC,27,42,2017
5,2017090700,00-0023436,KC,NE,A.Smith,2,2,1.000000,4.500000,4.500000,0,0,-0.484926,0.500000,-0.242463,0.182730,0.000000,0.000000,0.000000,0.000000,-0.242463,-0.018978,-0.009489,0.500000,-0.009489,0.138241,-0.013611,-0.006806,-0.006806,2017-09-07,NE,KC,27,42,2017
6,2017090700,00-0028087,NE,KC,D.Lewis,2,2,1.000000,4.500000,4.500000,0,0,-0.262034,0.000000,-0.131017,0.000000,0.000000,0.000000,0.000000,0.000000,-0.131017,-0.008146,-0.004073,0.000000,-0.004073,0.000000,-0.001089,-0.000544,-0.000544,2017-09-07,NE,KC,27,42,2017
7,2017090700,00-0033040,KC,NE,T.Hill,2,2,1.000000,2.500000,2.500000,0,0,-0.784858,0.000000,-0.392429,0.000000,0.000000,0.000000,0.000000,0.000000,-0.392429,-0.023383,-0.011692,0.000000,-0.011692,0.000000,-0.009410,-0.004705,-0.004705,2017-09-07,NE,KC,27,42,2017
8,2017090700,00-0030506,KC,NE,T.Kelce,1,1,1.000000,4.000000,4.000000,0,0,-0.059010,0.000000,-0.059010,0.000000,0.000000,0.000000,0.000000,0.000000,-0.059010,-0.000907,-0.000907,0.000000,-0.000907,0.000000,-0.000054,-0.000054,-0.000054,2017-09-07,NE,KC,27,42,2017
9,2017090700,00-0030668,KC,NE,C.West,1,1,1.000000,21.000000,21.000000,0,1,3.093750,1.000000,3.093750,1.000000,1.000000,0.000000,0.000000,1.000000,3.093750,0.024962,0.024962,1.000000,0.024962,1.000000,0.077226,0.077226,0.077226,2017-09-07,NE,KC,27,42,2017


In [40]:
list(zip(list(rf.predict(X_test_num.fillna(0))), list(y_test['Total_Yards'])))

[(148.40000000000001, 148),
 (45.0, 45),
 (38.100000000000001, 38),
 (17.0, 17),
 (15.0, 15),
 (9.0, 9),
 (9.0, 9),
 (5.0, 5),
 (4.0, 4),
 (21.0, 21),
 (4.0, 4),
 (110.2, 110),
 (42.299999999999997, 42),
 (22.0, 22),
 (15.9, 16),
 (41.0, 41),
 (1.0, 1),
 (51.600000000000001, 52),
 (37.200000000000003, 37),
 (16.0, 16),
 (66.400000000000006, 66),
 (13.0, 13),
 (7.0, 7),
 (72.0, 72),
 (80.0, 80),
 (9.0999999999999996, 9),
 (40.0, 40),
 (27.0, 27),
 (4.0, 4),
 (2.0, 2),
 (2.0, 2),
 (33.799999999999997, 35),
 (32.0, 32),
 (17.0, 17),
 (11.0, 11),
 (7.0, 7),
 (30.399999999999999, 30),
 (23.699999999999999, 23),
 (22.0, 22),
 (10.0, 10),
 (15.0, 15),
 (2.0, 2),
 (15.0, 15),
 (-1.0, -1),
 (10.0, 10),
 (21.0, 21),
 (100.09999999999999, 100),
 (65.0, 65),
 (42.0, 42),
 (8.0, 8),
 (16.0, 16),
 (0.0, 0),
 (15.0, 15),
 (4.0, 4),
 (76.0, 76),
 (44.100000000000001, 44),
 (25.0, 25),
 (22.0, 22),
 (26.0, 26),
 (4.0, 4),
 (5.0, 5),
 (3.0, 3),
 (45.899999999999999, 46),
 (29.699999999999999, 30),
 (30.